In [ ]:
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
import scipy.stats as st
%matplotlib qt

In [ ]:
def linear_slew(a, b, steps):
    slope = (b - a) / steps
    return a + slope * np.arange(steps)

def quadratic_slew(a, b, steps):
    t = np.arange(steps)

    # Standard quadratic form, assuming that we start
    # at the bottom of the parabola and increase thereafter
    x1 = 0
    x2 = t[-1]
    y2 = b
    y1 = a

    a = (y2 - y1) / (x2 - x1)**2
    b = -2 * a * x1
    c = y1 + a * x1**2

    return a*t**2 + b*t + c

total_time = 20 << u.s
time_bin = 0.1 << u.s

time_bins = np.arange(0, total_time.value + time_bin.value, time_bin.value) << u.s
time_mids = time_bins[:-1] + np.diff(time_bins)/2

num_time_bins = time_mids.size

temp_start, temp_end = (20, 24) << u.MK
em_start, em_end = (5, 8) << (1e47 * u.cm**-3)

In [ ]:
temps = linear_slew(temp_start, temp_end, num_time_bins)
ems = quadratic_slew(em_start, em_end, num_time_bins)

In [ ]:
import astropy.visualization as viz

fig, ax = plt.subplots()
with viz.quantity_support():
    ax.stairs(temps, time_bins.value, label='temperature', color='orange')
    axx = ax.twinx()
    axx.stairs(ems, time_bins.value, label='emission measures', color='blue')

ax.legend(loc='upper left')
axx.legend(loc='lower left')
plt.show()

In [ ]:
def spike(t, location, duration, height):
    slice_ = (t >= (location - duration/2)) & (t < (location + duration/2))

    def spike(tt):
        arg1 = (tt < duration / 2) * tt * (2 * height / duration)
        arg2 = (tt >= duration / 2) * 2 * height * (1 - tt / duration)
        return arg1 + arg2

    output = spike((t[1] - t[0]) * np.arange((slice_.sum())))
    ret = np.zeros_like(t)
    ret[slice_] = output
    return ret

In [ ]:
def noise_psd(N, psd = lambda f: 1):
        X_white = np.fft.rfft(np.random.randn(N))
        S = psd(np.fft.rfftfreq(N))
        # Normalize S
        S = S / np.sqrt(np.mean(S**2))
        X_shaped = X_white * S
        return np.fft.irfft(X_shaped)

def PSDGenerator(f):
    return lambda N: noise_psd(N, f)

@PSDGenerator
def white_noise(f):
    return 1

@PSDGenerator
def blue_noise(f):
    return np.sqrt(f)

@PSDGenerator
def violet_noise(f):
    return f

@PSDGenerator
def brownian_noise(f):
    return 1/np.where(f == 0, float('inf'), f)

@PSDGenerator
def pink_noise(f):
    return 1/np.where(f == 0, float('inf'), np.sqrt(f))

In [ ]:
# figure out the spectral slope time shape

# dominate this with a linear shift from high to low electron index
# aka a soft to hard spectrum
index_start, index_end = 8, 3

# we have particle injections across the interval
# which occur in blips where the index hardens
# significantly
num_injections = 10

# triangular injection rise/fall within this time
injection_duration = 1 << u.s

# how much harder does the spectrum become per injection,
# as a proportion?
injection_index_change = 0.2

# Deterministic rng, if we want
rng = np.random.default_rng()
# rng = np.random.default_rng(np.random.MT19937(1275830))
injection_time_indices = rng.integers(0, num_time_bins - 1, size=num_injections)

# The spectral index starts linearly going up
indices = np.abs(pink_noise(time_mids.size)) + linear_slew(index_start, index_end, num_time_bins)

# The electron flux goes up and then back down
efluxes = np.abs(pink_noise(time_mids.size))
efluxes += 1

# The spectral indices and electron fluxes spike
# at the same time: beamlike injection of electrons
for it in injection_time_indices:
    cur_index = indices[it] 
    indices -= (s := spike(
        time_mids.to_value(u.s),
        time_mids[it].to_value(u.s),
        injection_duration.to_value(u.s),
        injection_index_change * cur_index
    ))

    cur_flux = efluxes[it]
    print(cur_flux)
    injection_flux_change = 1
    efluxes += spike(
        time_mids.to_value(u.s),
        time_mids[it].to_value(u.s),
        injection_duration.to_value(u.s),
        injection_flux_change * cur_flux
    )

efluxes -= 1

In [ ]:
fig, ax = plt.subplots()
ax.stairs(indices, time_bins.value, label='spectral index')
ax.stairs(efluxes, time_bins.value, label='electron flux')
ax.legend()
plt.show()

In [ ]:
cutoff_energies = (20 << u.keV) * np.ones_like(time_mids.value)

In [ ]:
from yaff import common_models as cm
from yaff.fitting import Parameter

def model(params: cm.ArgsT):
    return cm.thermal(params) + cm.thick_target(params)

In [ ]:
energy_bins = np.geomspace(4, 200, num=60) << u.keV
area = 10 << u.cm**2
exposure = total_time
de = np.diff(energy_bins) << u.keV
energy_mids = (energy_bins[:-1] + np.diff(energy_bins)/2)

In [ ]:
thermal_truth = np.zeros((time_mids.size, energy_bins.size - 1)) << u.ph
nonthermal_truth = np.zeros((time_mids.size, energy_bins.size - 1)) << u.ph

for i in range(cutoff_energies.size):
    parameters = {
        'temperature': temps[i],
        'emission_measure': ems[i],
        'cutoff_energy': cutoff_energies[i],
        'spectral_index': indices[i] << u.one,
        'electron_flux': (float(efluxes[i]) << (1e35 * u.electron / u.s))
    }
    parameters = {k: Parameter(v, False) for (k, v) in parameters.items()}
    args = {
        'parameters': parameters,
        'photon_energy_edges': energy_bins.to_value(u.keV)
    }
    thermal_truth[i] = (cm.thermal(args) << (u.ph / u.cm**2 / u.keV / u.s)) * time_bin * area * de
    nonthermal_truth[i] = (cm.thick_target(args) << (u.ph / u.keV / u.cm**2 / u.s)) * time_bin * area * de

thermal_truth[thermal_truth < 0] = 0
nonthermal_truth[nonthermal_truth < 0] = 0

def full_spectrogram():
    return thermal_truth + nonthermal_truth

In [ ]:
import matplotlib.colors as mcol

unit = u.ph

fig, ax = plt.subplots()
norm = mcol.SymLogNorm(linthresh=1, vmin=0, vmax=full_spectrogram().to_value(unit).max())
cmap = plt.get_cmap('plasma').copy()
ax.pcolormesh(
    time_bins.to_value(u.s),
    energy_bins,
    full_spectrogram().T.to_value(unit),
    norm=norm,
    cmap=cmap
)
ax.set(yscale='log', xlabel='time (s)', ylabel='energy (keV)')
plt.show()

In [ ]:
test_idxs = (10, 20)
with viz.quantity_support():
    fig, ax = plt.subplots()
    for i in test_idxs:
        th_example = thermal_truth[i]
        nth_example = nonthermal_truth[i]
        ret = ax.stairs(th_example, energy_bins)
        col = ret.get_edgecolor()
        ax.stairs(nth_example, energy_bins, color=col)
        ax.set(xscale='log', yscale='log', xlabel='energy keV')
plt.show()

In [ ]:
closest = lambda a, v: np.argmin(np.abs(a - v))

'''
Let's say we have a Ba133 source on board.
For X-rays that's about 2e5 count/second.
There will be lines at 4 keV, 31 keV, and 81 keV
'''
count_rate = 10
noise = (count_rate * num_time_bins * time_bin) * np.ones_like(thermal_truth[0].value)
noise[closest(energy_mids, 4 << u.keV):closest(energy_mids, 6 << u.keV)] *= 100
noise[closest(energy_mids, 29 << u.keV):closest(energy_mids, 32 << u.keV)] *= 10
noise[closest(energy_mids, 80 << u.keV):closest(energy_mids, 84 << u.keV)] *= 5

background = (st.norm.rvs(loc=noise, scale=np.sqrt(noise), size=(time_bins.size - 1, noise.size)).T / num_time_bins) * u.ph
background *= 0

thermal_photons = thermal_truth
nonthermal_photons = nonthermal_truth
# photons += (background.T.to_value(u.one) << u.ph)
thermal_photons = thermal_photons.astype(int)
nonthermal_photons = nonthermal_photons.astype(int)

systematic = 0.05
data = st.norm.rvs(
    loc=(tot := (thermal_photons + nonthermal_photons + background.T).to_value(u.ph)),
    scale=np.sqrt(tot + (systematic * tot)**2).astype(int)
) << u.ph

data[data < 0] = 0

In [ ]:
import matplotlib.colors as mcol

unit = u.ph

fig, ax = plt.subplots()
norm = mcol.LogNorm()
cmap = plt.get_cmap('plasma').copy()
ax.pcolormesh(
    time_bins.to_value(u.s),
    energy_bins,
    data.T.to_value(unit),
    norm=norm,
    cmap=cmap
)
ax.set(yscale='log', xlabel='time (s)', ylabel='energy (keV)')
plt.show()

In [ ]:
fig, ax = plt.subplots()
with viz.quantity_support():
    ax.stairs(data.sum(axis=0), energy_bins, label='data')
    ax.stairs(thermal_photons.sum(axis=0), energy_bins, label='thermal')
    ax.stairs(nonthermal_photons.sum(axis=0), energy_bins, label='thermal')
    ax.stairs(background.sum(axis=1), energy_bins, label='bg')
    ax.set(xscale='log', yscale='log')
    ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()
for s in data.T:
    ax.stairs(s.value, time_bins.value)
ax.set(yscale='log', ylim=(1e-9, 1e6))
plt.show()

In [ ]:
from tedec import decomp

In [ ]:
dimensionless_data = data.to_value(u.ph).T.astype(int)

nearest = lambda a, v: np.argmin(np.abs(a - v))

ta, tb = nearest(energy_mids, 5 << u.keV), nearest(energy_mids, 6 << u.keV)
na, nb = nearest(energy_mids, 31 << u.keV), nearest(energy_mids, 33 << u.keV)

pack = decomp.DataPacket(
    data=dimensionless_data,
    basis_timeseries=[
        dimensionless_data[ta:tb].sum(axis=0),
        dimensionless_data[na:nb].sum(axis=0),
    ],
    constant_offset=False
)

systematic = 0.1
ret = decomp.bootstrap(
    pack,
    errors=np.sqrt(dimensionless_data + (systematic * dimensionless_data)**2),
    num_iter=1000
)

In [ ]:
from yaff import plotting
from astropy import visualization as viz

sys = 0.05
err = lambda a: np.sqrt(a + (a * sys)**2)

th_mean = ret[:, 0, :].mean(axis=0) << u.ph
th_std = ret[:, 0, :].std(axis=0) << u.ph
nth_mean = ret[:, 1, :].mean(axis=0) << u.ph
nth_std = ret[:, 1, :].std(axis=0) << u.ph

# scale by # time bins (need to update)
# bkg_part = ret[:, 2, :] * (num_time_bins - 1)
bkg_part = ret[:, 1, :] * 0
bkg_mean = bkg_part.mean(axis=0) * u.ph
bkg_std = bkg_part.std(axis=0) * u.ph

fig, ax = plt.subplots()

# thermal_ph = (thermal_truth.sum(axis=0) * time_bin * area  * de).to_value(u.ph)
# nonthermal_ph = (nonthermal_truth.sum(axis=0) * time_bin * area * de).to_value(u.ph)

with viz.quantity_support():
    # plotting.stairs_with_error(energy_bins, th := thermal_photons.sum(axis=0), err(th.value) << u.ph, label='true thermal spectrum')
    # plotting.stairs_with_error(energy_bins, nth := nonthermal_photons.sum(axis=0), err(nth.value) << u.ph, label='true nonthermal spectrum')
    plotting.stairs_with_error(energy_bins, dm := dimensionless_data.sum(axis=1) << u.ph, err(dm.value) << u.ph, label='full spectrum')
    # ax.stairs(background.sum(axis=1), energy_bins, label='true background')
    
    num_sigma = 1
    plotting.stairs_with_error(energy_bins << u.keV, th_mean << u.ph, num_sigma*th_std, ax=ax, label='decomposed thermal')
    plotting.stairs_with_error(energy_bins << u.keV, nth_mean << u.ph, num_sigma*nth_std, ax=ax, label='decomposed nonthermal')
    plotting.stairs_with_error(
        energy_bins << u.keV,
        (nth_mean + th_mean) << u.ph,
        num_sigma*np.sqrt(nth_std**2 + th_std**2),
        ax=ax,
        label='decomposed full'
    )

ax.legend()
ax.set(xscale='log', yscale='log', ylim=(None, None))
plt.show()

In [ ]:
dimensionless_data.sum(), (th_mean + nth_mean).sum()